# This notebook was not used for the paper due to questionable ground truth of the gittables dataset -> benchmark was used instead

In [ ]:
# Notebook to extract and predict gittables dataset

In [1]:
#Reload modules before executing code
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from os.path import join
from os import listdir
import json
from re import sub #for camel case conversion

import numpy as np
import pandas as pd
from pyarrow.parquet import ParquetFile
from sklearn.metrics import f1_score, classification_report

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
# from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
#     convert_string_lists_to_lists,
    prepare_feature_extraction,
#     load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

### Utils

In [3]:
def camel_case(s):
    try:
      c = sub(r"(_|-)+", " ", s).title().replace(" ", "")
      return ''.join([c[0].lower(), c[1:]])
    except:
#         print(f'Error converting {s} to camel case')
        return ''

In [4]:
# !! Make sure you downloaded a gittables dataset (f.e. abstraction_tables)
# and unzipped the data into the directory mentioned below !!
path = '../data/data/gittables/whole_tables'

with open(join(path, '../../../types.json'), 'r') as f:  
        types = json.load(f)['type78']

filepaths = [join(path, f) for f in listdir(path) if f.endswith('.parquet')]

In [5]:
# filepaths
# types

In [45]:
columns = []
col_types = []
col_ids = []
curr_id = 0
start = datetime.now()
print(f'Started at {start}')

for fp in filepaths[:5000]:
    try:
        table = pd.read_parquet(fp)
        table.rename(columns=camel_case,inplace=True)
    except:
        print(f'Error converting table in {fp}')
        continue
    for idx, col_name in enumerate(table.columns):
        if len(col_name) == 0: #no type defined, skip col
            continue
        if col_name not in types:
            continue
        col = table.iloc[:, idx]
        try: #TODO fix 2 columns in table with same name
            columns.append(list(map(str, col.to_list())))
            col_types.append(col_name)
            col_ids.append(curr_id)
            curr_id+=1
        except:
            print(f'error in table {fp}')
#             for name,c in col.items():
#                 print(c)
#                 columns.append(list(map(str, c.to_list())))
#                 col_types.append(col_name)
#                 col_ids.append(curr_id)
#                 curr_id+=1
        
        
print(f'Finished loading tables at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-03-28 12:35:57.808264
Finished loading tables at 2022-03-28 12:36:19.401741, took 0:00:21.593485 seconds


In [46]:
len(set(col_types))

72

In [47]:
len(col_types)

2508

### Init feature extraction models

In [48]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:02.766531 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.870848 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.000662 seconds.


[nltk_data] Downloading package punkt to /home/senn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/senn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
data = pd.Series(
    [
        *columns #expand/unpack the list
    ],
    name="values"
)

In [50]:
data.head(5)

0    [Gear\nBags, Gear\nBags\nCollections\nErin Rec...
1    [Joust Duffle Bag, Strive Shoulder Pack, Crown...
2    [<p>The sporty Joust Duffle Bag can't be beat ...
3                                                   []
4                                                   []
Name: values, dtype: object

In [51]:
extract_features(
    join(path, "processed/temporary.csv"),
    data
)
feature_vectors = pd.read_csv(join(path, "processed/temporary.csv"), dtype=np.float32)

Extracting Features:   0%|                                                              | 0/2508 [00:00<?, ?it/s]

Exporting 1588 column features


Extracting Features: 100%|██████████████████████████████████████████████████| 2508/2508 [00:22<00:00, 110.70it/s]


In [52]:
# feature_vectors.head()

### Init sherlock

In [53]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

In [54]:
predicted_labels = model.predict(feature_vectors, "sherlock")

### Analysing results

In [55]:
size = len(predicted_labels)
print(f'prediction count {size}')


# Should be fully deterministic too.
f1_score(col_types[:size], predicted_labels[:size], average="weighted")

prediction count 2508


0.4518698024776634

### All Scores

In [56]:
print(classification_report(col_types[:size], predicted_labels[:size], digits=4))

                precision    recall  f1-score   support

       address     0.0205    0.7692    0.0399        26
   affiliation     0.0000    0.0000    0.0000         0
           age     0.7273    0.3478    0.4706        46
         album     0.4211    0.6667    0.5161        12
          area     0.0000    0.0000    0.0000        20
        artist     0.6615    0.7679    0.7107        56
   birth Place     0.0000    0.0000    0.0000         0
         brand     0.5000    0.1818    0.2667        11
      capacity     0.0000    0.0000    0.0000         9
      category     0.6000    0.3000    0.4000        60
          city     0.5000    0.3333    0.4000        48
         class     0.0690    0.0588    0.0635        34
classification     0.0000    0.0000    0.0000         6
          club     0.0000    0.0000    0.0000         9
          code     0.2857    0.0667    0.1081        30
    collection     0.0000    0.0000    0.0000        23
       command     0.0000    0.0000    0.0000  

/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/v

In [57]:
model_id = "sherlock"

classes = np.load(f"../model_files/classes_{model_id}.npy", allow_pickle=True)

report = classification_report(col_types[:size], predicted_labels[:size], output_dict=True)

class_scores = list(filter(lambda x: isinstance(x, tuple) and isinstance(x[1], dict) and 'f1-score' in x[1] and x[0] in classes, list(report.items())))

class_scores = sorted(class_scores, key=lambda item: item[1]['f1-score'], reverse=True)

/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/v

### Top 5 types

In [58]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[0:5]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
year		0.908		0.994		0.836		207
type		0.891		0.972		0.822		510
industry	0.767		0.943		0.647		51
artist		0.711		0.662		0.768		56
rank		0.667		0.654		0.680		50


In [59]:
### Bottom 5 types

In [60]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[len(class_scores)-5:len(class_scores)]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
result		0.000		0.000		0.000		41
sales		0.000		0.000		0.000		7
service		0.000		0.000		0.000		18
team		0.000		0.000		0.000		28
team Name	0.000		0.000		0.000		0


In [61]:
type_freq_df = pd.DataFrame(col_types, columns=['type'])
type_freq_df = pd.DataFrame(type_freq_df['type'].value_counts())
type_freq_df.columns = ['count']
type_freq_df.index.name = 'type'


In [62]:
# type_freq_df

In [63]:
alt.Chart(type_freq_df.reset_index()).mark_bar(size=8).encode(
    x = alt.X('type:O',
              title = 'Semantic Types',
              sort=alt.EncodingSortField(
                field="count",  
                order="descending")),
    y = alt.Y('count', title='Number of Samples')    
).properties(width=800,height=200)

alt.Chart(...)